#  Market forecasting for car prices

Acquisition

Let's start with imports: Pandas will help us handling DataFrames, and BeautifulSoup will help us with the parsing of the files.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
def soup_object_creation(url):
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(20)
        driver.maximize_window()
        time.sleep(4)
        
        feed_list = driver.find_element(By.CSS_SELECTOR, "div.feed_list")
        time.sleep(2)
        
        car_ad_elements = feed_list.find_elements(By.CSS_SELECTOR, 'div.feeditem.table[data-v-3ecba55a=""]')
        
        for ad_element in car_ad_elements:
            try:
                time.sleep(4)
                ad_element.click()
            except:
                pass
        
        html_content = driver.page_source
        
        soup = BeautifulSoup(html_content, 'html.parser')
        return soup
    except Exception as e:
        print(e)

In [ ]:
ad_columns = ['car_title', 'sub_model', 'year_of_manufactoring', 'num_of_owners', 'car_engine_size', 'price']
# 'num_of_kilometers', 'engine_type', 'gear_box', 'car_color', 'test_valid_until', 'current_ownership','former_ownership', 'suits_for_disabled', 'rate_of_pollution', 'region'
df = pd.DataFrame(columns=ad_columns)

In [ ]:
def fetch_data(soup, df):
    # Defining the raw strings for the tags' names
    item_id_raw = 'feed_item_{}'
    car_title_raw = 'feed_item_{}_title'
    data_year_raw = 'data_year_{}'
    num_of_owners_raw = 'data_hand_{}'
    car_engine_size_raw = 'data_engine_size_{}'
    price_raw = 'feed_item_{}_price'
    
    # Iterating over 36 adverts that appear in the page
    for i in range(36):
        temp_dict_to_append = {}

        current_id = item_id_raw.format(i)
        current_title_id = car_title_raw.format(i)
        current_year_id = data_year_raw.format(i)
        current_num_of_owners=num_of_owners_raw.format(i)
        current_car_engine_size = car_engine_size_raw.format(i)
        current_price = price_raw.format(i)
                
        # Updating the dictionary
        try:
            temp_dict_to_append['car_title'] = soup.find('span', {'id': current_title_id}).find('span', class_='title').get_text().strip()
        except:
            temp_dict_to_append['car_title'] = None
        try:
            temp_dict_to_append['sub_model'] = soup.find('div', {'id': current_id}).find('span', class_='subtitle').get_text()
        except:
            temp_dict_to_append['sub_model'] = None
        try:
            temp_dict_to_append['year_of_manufactoring'] = soup.find('span', {'id': current_year_id}).get_text()
        except:
            temp_dict_to_append['year_of_manufactoring'] = None
        try:
            temp_dict_to_append['num_of_owners'] = soup.find('span', {'id': current_num_of_owners}).get_text()
        except:
            temp_dict_to_append['num_of_owners'] = None
        try:
            temp_dict_to_append['car_engine_size'] = soup.find('span', {'id': current_car_engine_size}).get_text()
        except:
            temp_dict_to_append['car_engine_size'] = None
        try:
            temp_dict_to_append['price'] = soup.find('div', {'id': current_price}).get_text().strip().split(' ')[0].replace(',', '')
        except:
            temp_dict_to_append['price'] = None
        try:
            temp_dict_to_append['air_pollution_rate'] = soup.find('div', {'id': current_id}).find('div', class_='level_color selected').get_text().strip()
        except:
            temp_dict_to_append['air_pollution_rate'] = None
        
        # Taking care of the dynamic fields and updating the dictionary accordingly
        temp_details_wrapper = soup.find('div', {'id': current_id}).find('div', attrs={'data-v-27ef3d9f': True, 'data-v-212a6bed': True})
        
        try:
            temp_dl_tags_array = temp_details_wrapper.find_all('dl')
            for detail in temp_dl_tags_array:
                dt_tag = detail.find('dt').get_text().strip()
                dd_tag = detail.find('dd').find('span').get_text().strip()
                temp_dict_to_append[dt_tag] = dd_tag
        except:
            continue
        
        # For debugging purposes
        print(temp_dict_to_append)
        
        # Converting the dictionary to an DataFrame so we can append it to the original one
        dict_as_pd = pd.DataFrame(temp_dict_to_append, index=[0])
        df = pd.concat([df, dict_as_pd], ignore_index=True)
    return df

In [ ]:
def crawling_yad_2(df):
    base_url_to_adjust = 'https://www.yad2.co.il/vehicles/cars?priceOnly=1&page={}'
    temp = 2
    
    while temp <= 220:
        temp_url = base_url_to_adjust.format(temp)
        soup = soup_object_creation(temp_url)
        df = fetch_data(soup, df)
        time.sleep(4)
        print('Scraped page number {}'.format(temp))
        temp+=1
    
    return df

In [ ]:
df = crawling_yad_2(df)

In [ ]:
df.to_csv('concat_df.csv', encoding='utf-8', index=False)